In [375]:
import numpy as np
import glob as glob
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from utils import reject_sampling,subseting_samples
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

FOLDER_PATH = './Data/'
file_paths = glob.glob(FOLDER_PATH + '/*')
file_paths=sorted(file_paths)

image_data=[];labels=[]
for file_path in file_paths:
    image = Image.open(file_path)
    label=file_path.split('/')[-1].split('.bmp')[0][0]
    image_data.append(np.array(image))
    labels.append(label)

print(len(image_data)==len(labels))
data=np.vstack([row.reshape(-1) for row in image_data])
A = data[:75,:].T
B = data[75*1:75*2,:].T
C = data[75*2:75*3,:].T
D = data[75*3:75*4,:].T
E = data[75*4:75*5,:].T
F = data[75*5:75*6,:].T
G = data[75*6:75*7,:].T
H = data[75*7:75*8,:].T
I = data[75*8:75*9,:].T
J = data[75*9:75*10,:].T
K = data[75*10:75*11,:].T
L = data[75*11:75*12,:].T
M = data[75*12:75*13,:].T
#We only choose subet, cause in prcatice, no that much data
SUB_NUM=20
A=subseting_samples(A,SUB_NUM)
B=subseting_samples(B,SUB_NUM)
C=subseting_samples(C,SUB_NUM)
D=subseting_samples(D,SUB_NUM)
E=subseting_samples(E,SUB_NUM)
F=subseting_samples(F,SUB_NUM)
G=subseting_samples(G,SUB_NUM)
H=subseting_samples(H,SUB_NUM)
I=subseting_samples(I,SUB_NUM)
J=subseting_samples(J,SUB_NUM)
K=subseting_samples(K,SUB_NUM)
L=subseting_samples(L,SUB_NUM)
M=subseting_samples(M,SUB_NUM)
print(A.shape==M.shape)
print(A.shape)

True
True
(4096, 20)


In [376]:
NUM_SAMPLE=20
A_samples = reject_sampling(A,NUM_SAMPLE)
B_samples = reject_sampling(B,NUM_SAMPLE)
C_samples = reject_sampling(C,NUM_SAMPLE)
D_samples = reject_sampling(D,NUM_SAMPLE)
E_samples = reject_sampling(E,NUM_SAMPLE)
F_samples = reject_sampling(F,NUM_SAMPLE)
G_samples = reject_sampling(G,NUM_SAMPLE)
H_samples = reject_sampling(H,NUM_SAMPLE)
I_samples = reject_sampling(I,NUM_SAMPLE)
J_samples = reject_sampling(J,NUM_SAMPLE)
K_samples = reject_sampling(K,NUM_SAMPLE)
L_samples = reject_sampling(L,NUM_SAMPLE)
M_samples = reject_sampling(M,NUM_SAMPLE)

In [377]:
X = np.concatenate([A,B,C,D,E,
                    F,G,H,I,J,
                    K,L,M,A_samples.T, B_samples.T, C_samples.T, D_samples.T, E_samples.T,
                    F_samples.T, G_samples.T, H_samples.T, I_samples.T, J_samples.T,
                    K_samples.T, L_samples.T, M_samples.T],axis=1).T


y = np.array(['A'] * A.shape[1] + ['B'] * B.shape[1] +
             ['C'] * C.shape[1] + ['D'] * D.shape[1] +
             ['E'] * E.shape[1] + ['F'] * F.shape[1] +
             ['G'] * G.shape[1] + ['H'] * H.shape[1] +
             ['I'] * I.shape[1] + ['J'] * J.shape[1] +
             ['K'] * K.shape[1] + ['L'] * L.shape[1] +
             ['M'] * M.shape[1]+['A'] * len(A_samples) + ['B'] * len(B_samples) +
             ['C'] * len(C_samples) + ['D'] * len(D_samples) +
             ['E'] * len(E_samples) + ['F'] * len(F_samples) +
             ['G'] * len(G_samples) + ['H'] * len(H_samples) +
             ['I'] * len(I_samples) + ['J'] * len(J_samples) +
             ['K'] * len(K_samples) + ['L'] * len(L_samples) +
             ['M'] * len(M_samples))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

In [378]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(416, 4096)
(416,)
(104, 4096)
(104,)


In [379]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = X_train.shape[1]
hidden_size = 400
output_size = len(label_encoder.classes_)
model = MLP(input_size, hidden_size, output_size).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_accuracy = correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}, Train Accuracy: {train_accuracy}")


model.eval()
with torch.no_grad():
    X_test_tensor, y_test_tensor = X_test_tensor.to(device), y_test_tensor.to(device)
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_test_tensor).sum().item()
    total = len(y_test_tensor)
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy}")


Epoch 1/10, Loss: 1444.5408325195312, Train Accuracy: 0.36538461538461536
Epoch 2/10, Loss: 440.4207534790039, Train Accuracy: 0.6466346153846154
Epoch 3/10, Loss: 207.89177417755127, Train Accuracy: 0.7427884615384616
Epoch 4/10, Loss: 33.29121422767639, Train Accuracy: 0.8701923076923077
Epoch 5/10, Loss: 10.55099630355835, Train Accuracy: 0.9447115384615384
Epoch 6/10, Loss: 1.483999768963205, Train Accuracy: 0.9831730769230769
Epoch 7/10, Loss: 2.712982813894996, Train Accuracy: 0.9831730769230769
Epoch 8/10, Loss: 2.19846148416247, Train Accuracy: 0.9759615384615384
Epoch 9/10, Loss: 0.389125603712273, Train Accuracy: 0.9951923076923077
Epoch 10/10, Loss: 1.0114516756875667, Train Accuracy: 0.9927884615384616
Test Accuracy: 0.9807692307692307


# No sampling what if

In [380]:
X = np.concatenate([A,B,C,D,E,
                    F,G,H,I,J,
                    K,L,M],axis=1).T


y = np.array(['A'] * A.shape[1] + ['B'] * B.shape[1] +
             ['C'] * C.shape[1] + ['D'] * D.shape[1] +
             ['E'] * E.shape[1] + ['F'] * F.shape[1] +
             ['G'] * G.shape[1] + ['H'] * H.shape[1] +
             ['I'] * I.shape[1] + ['J'] * J.shape[1] +
             ['K'] * K.shape[1] + ['L'] * L.shape[1] +
             ['M'] * M.shape[1])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

In [381]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(208, 4096)
(208,)
(52, 4096)
(52,)


In [382]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder


class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)


train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = X_train.shape[1]
hidden_size = 400
output_size = len(label_encoder.classes_)
model = MLP(input_size, hidden_size, output_size).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_accuracy = correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}, Train Accuracy: {train_accuracy}")


model.eval()
with torch.no_grad():
    X_test_tensor, y_test_tensor = X_test_tensor.to(device), y_test_tensor.to(device)
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == y_test_tensor).sum().item()
    total = len(y_test_tensor)
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy}")


Epoch 1/10, Loss: 1604.2152252197266, Train Accuracy: 0.08173076923076923
Epoch 2/10, Loss: 2584.6950073242188, Train Accuracy: 0.1201923076923077
Epoch 3/10, Loss: 2521.4771728515625, Train Accuracy: 0.11538461538461539
Epoch 4/10, Loss: 1327.0437927246094, Train Accuracy: 0.19230769230769232
Epoch 5/10, Loss: 493.94085693359375, Train Accuracy: 0.15384615384615385
Epoch 6/10, Loss: 243.07913970947266, Train Accuracy: 0.28365384615384615
Epoch 7/10, Loss: 157.50301265716553, Train Accuracy: 0.25961538461538464
Epoch 8/10, Loss: 43.88104295730591, Train Accuracy: 0.2932692307692308
Epoch 9/10, Loss: 28.301241636276245, Train Accuracy: 0.39903846153846156
Epoch 10/10, Loss: 19.695741891860962, Train Accuracy: 0.5721153846153846
Test Accuracy: 0.40384615384615385
